In [1]:
import winsound
import pygame
from bs4 import BeautifulSoup, NavigableString
import requests
from time import sleep
from random import randint
import math
from tqdm import tqdm
import pickle
import pandas as pd
from datetime import date
from datetime import datetime
import json

import pymysql                        # for getting data from a SQL database
from sqlalchemy import create_engine, text  # for establishing the connection and authentication
import zlib

from webscraper.scrap_modules import my_NLP as myNLP 
from webscraper.scrap_modules import scraper_ds as mybib
from webscraper.scrap_modules import prep as myPrep

%load_ext autoreload
%autoreload 2

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


### load scraped raw data

In [2]:

with open("webscraper/webscrap_data/soup_dict.pkl", "rb") as f:
    scraper_dict = pickle.load(f)
    
with open("webscraper/webscrap_data/keyword_dict.pkl", "rb") as f:
    keyword_dict = pickle.load(f)

In [3]:
len(scraper_dict)

22079

### transform scraped raw data into df 

In [5]:
df = myPrep.transform_scrap_data_to_df(scraper_dict)

100%|██████████| 22079/22079 [12:54<00:00, 28.51it/s]


In [ ]:
# df.head(5)

,id,title,company,city,posting_date,job_description,seniority_level,employment_type,job_function,industries,scraping_date,url
0,3685669741,Software Engineer Fullstack (m/w/d),Atruvia AG,"Aschheim, Bavaria, Germany",1 day ago,VollzeitWir sind der Digitalisierungspartner d...,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
1,3719620327,Data Analyst - Business Intelligence,Almedia,"Berlin, Berlin, Germany",5 days ago,Almedia helps companies grow by promoting thei...,Mid-Senior level,Full-time,Analyst,"Technology, Information and Internet",2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
2,3720365801,(Junior) Data Engineer (m/f/d),Sandbox Interactive,"Berlin, Berlin, Germany",1 day ago,YOUR GAMEBuild and maintain the current data i...,Mid-Senior level,Full-time,Information Technology,Computer Games,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
3,3672508925,Financial Data Analyst (m/f/d),Ultramarin,"Berlin, Berlin, Germany",2 months ago,About The PositionWe are looking for a top-tie...,Mid-Senior level,Full-time,Information Technology,Financial Services,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
4,3695466781,Business Intelligence Analyst (f/m/d),Les Lunes,"Berlin, Berlin, Germany",4 weeks ago,Your missionAs a Business Intelligence Analyst...,Mid-Senior level,Full-time,"Research, Analyst, and Information Technology",Retail Apparel and Fashion,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
5,3585738180,Data Analyst,Orange Quarter,"Berlin, Berlin, Germany",5 months ago,Orange Quarter are working with an exciting st...,Entry level,Full-time,Information Technology,Staffing and Recruiting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
6,3585734564,Data Analyst,Orange Quarter,"Berlin, Berlin, Germany",5 months ago,Orange Quarter are currently looking for a Dat...,Entry level,Full-time,Information Technology,Staffing and Recruiting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
7,3704916810,BI Analyst (f/m/d),Enter,"Berlin, Berlin, Germany",3 weeks ago,Build the future with Enter.Our mission is to ...,Mid-Senior level,Full-time,Information Technology,Architecture and Planning,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
8,3585734595,Data Analyst,Orange Quarter,"Berlin, Berlin, Germany",5 months ago,We are currently working with an emerging SaaS...,Entry level,Full-time,Information Technology,Staffing and Recruiting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
9,3698396795,Senior Data Analyst,Klim,"Berlin, Berlin, Germany",1 month ago,"We are Klim, a Berlin-based clean-tech company...",Mid-Senior level,Full-time,Other,IT Services and IT Consulting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...


In [11]:
df["title"].isna().sum()

275

### data cleaning - empty rows
Trying to fill none values doing a second request

In [ ]:
empty_id_list = empty_rows['id'].tolist()

for id in empty_id_list:
    print(f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{id}")

In [9]:
cleaned_df = myPrep.null_value_cleaner(df, 4) # increases each round the sleep time for scraping process

100%|██████████| 275/275 [00:06<00:00, 41.48it/s]


Round0: 7 empty values left


100%|██████████| 7/7 [00:00<00:00, 56.33it/s]

Round1: 0 empty values left
Finished in round <built-in function id>


In [ ]:
# display(cleaned_df.head(5))

,id,title,company,city,posting_date,job_description,seniority_level,employment_type,job_function,industries,scraping_date,url
0,3685669741,Software Engineer Fullstack (m/w/d),Atruvia AG,"Aschheim, Bavaria, Germany",1 day ago,VollzeitWir sind der Digitalisierungspartner d...,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
1,3719620327,Data Analyst - Business Intelligence,Almedia,"Berlin, Berlin, Germany",5 days ago,Almedia helps companies grow by promoting thei...,Mid-Senior level,Full-time,Analyst,"Technology, Information and Internet",2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
2,3720365801,(Junior) Data Engineer (m/f/d),Sandbox Interactive,"Berlin, Berlin, Germany",1 day ago,YOUR GAMEBuild and maintain the current data i...,Mid-Senior level,Full-time,Information Technology,Computer Games,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
3,3672508925,Financial Data Analyst (m/f/d),Ultramarin,"Berlin, Berlin, Germany",2 months ago,About The PositionWe are looking for a top-tie...,Mid-Senior level,Full-time,Information Technology,Financial Services,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
4,3695466781,Business Intelligence Analyst (f/m/d),Les Lunes,"Berlin, Berlin, Germany",4 weeks ago,Your missionAs a Business Intelligence Analyst...,Mid-Senior level,Full-time,"Research, Analyst, and Information Technology",Retail Apparel and Fashion,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...


### Add keywords

In [14]:
small_dict = {}
for key in keyword_dict:
    small_dict[key] = keyword_dict[key][0]

In [15]:
cleaned_df['keyword'] = cleaned_df['id'].map(small_dict)

### Add language 

In [19]:

cleaned_df = myNLP.add_language_values_to_df(cleaned_df, search_column = 'job_description') # add language values 


Processing: 100%|██████████| 22079/22079 [03:30<00:00, 104.73it/s]


### Save data

In [21]:
myPrep.save_cleaned_df(cleaned_df, 'cleaned_dataframe')